### Funcion que lee un fichero


In [268]:
def leer_fichero_distancia(anyo,filename):
    """Function that reads a file and creates a dictionary with the activity distance
    
    Args: name of the file to read
    
    Returns: a dictionary structure containing cycling activity data points
        """
    
    meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep','Oct', 'Nov', 'Dic']
    
    dic = {}
    with open(filename, mode='rt', encoding='utf-8') as f:
        for line in f:
#            print(line)
            l=line.split(',')
            if l[0] in meses:
#                print(l)
                mes = l[0]
               
                km =l[2]
                km2 = float(km[:-1])
#                print('En {m} has hecho -{k}-'.format(m=mes, k=km2))
            
                dic[mes] = km2
    return (anyo,dic)


### Funcion que lee todos los ficheros disponibles

In [269]:

def leer_todos_ficheros():
    """Function that reads all files availables and creates a dictionary with the activity distance for all years
    
    Args: None
    
    Returns: a dictionary structure wiht one entry per year containing cycling activity data points
        """
    dic_total={}

    (year,dic) = leer_fichero_distancia(2016,'ACTIVITY_TOTAL_DISTANCE 2016.csv')
    dic_total[year] =dic

    (year,dic) = leer_fichero_distancia(2017,'ACTIVITY_TOTAL_DISTANCE 2017.csv')
    dic_total[year] =dic

    (year,dic) = leer_fichero_distancia(2018,'ACTIVITY_TOTAL_DISTANCE 2018.csv')
    dic_total[year] =dic

    (year,dic) = leer_fichero_distancia(2019,'ACTIVITY_TOTAL_DISTANCE 2019.csv')
    dic_total[year] =dic
    
    return(dic_total)


### Funcion de ayuda: valor para un mes

In [270]:
def valor_mes(dictio, year, month):
    
    """Function that returns from a dictionary the value for an specific year and month
    
    Args:   dictionary - the dictionary wiht the data
            year - the year to retrieve the value from
            month - the month to retrieve the value from
    
    Returns: a float value wiht the distance for that year en month
    
    Exception: KeyError when wrong year or month
        """
    try:
        val= dictio [year][month]
        
    except KeyError as e:
        print('Issues in getting valor_mes: {error}.'.format(error=e))
        raise
    
    return val
        

### Funcion de ayuda: incremento para un mes

In [286]:
def inc_mensual(dictio, year, month):
    """Function that returns from a dictionary a year and a month the value and increment for that month
    
    Args:   dictionary - the dictionary with the data
            year - the year of the month
            month - the month to estite
    
    Returns: tuple with the values for
            total - total distance for the mongh
            increment - change in distance over previous month
            increment per cent - per cent change over previous month
    
    Exception: KeyError when wrong year or month
        """
    
    try:
        prev_year=year-1
        valor = valor_mes(dictio, year, month)
        
        if prev_year not in dictio:
            return(valor, valor, 100)
        else:
            valor_prev = valor_mes(dictio,prev_year, month)
          
            difabs = valor - valor_prev
            difpct= ((valor *100)/valor_prev)-100
            
            return(valor, difabs,difpct)
    
    except KeyError as e:
        print('Issues in getting inc_mensual: {error}.'.format(error=e))
        raise

### Funcion de ayuda: valor para un año

In [280]:
def valor_anyo(dictio, year):
    """Function that returns from a dictionary the value for an specific year 
    
    Args:   dictionary - the dictionary wiht the data
            year - the year to retrieve the value from
    
    Returns: a integer value wiht the instance for that year
    
    Exception: KeyError when wrong year or month
        """
    
#    dic_aux=dictio[year]
#    total=0
#    for month in dic_aux:
#        val=dic_aux[month]
#        total=total+val
#        print('{m} ---->{v}'.format(m=month,v=val))
#    return(total)
    try: 
        return int( sum([ dictio[year][month]  for month in dictio[year]]))
    
    except KeyError as e:
        print('Issues in getting valor_anyo: {error}.'.format(error=e))
        raise

### Funcion de ayuda: valor total

In [281]:
def valor_total(dictio):
    """Function that returns from a dictionary the total value for all years included
    
    Args:   dictionary - the dictionary with the data
    
    Returns: value wiht the instance for that year
    
    Exception: KeyError when wrong year or month
        """
    
    return  sum([valor_anyo(dictio,year) for year in dictio])

### Funcion de ayuda: incremento anual

In [287]:
def inc_anual(dictio, year):
  
    try:
        prev_year=year-1
        if prev_year not in dictio:
            return(valor_anyo(dictio,year),100)
        else:
            valor = valor_anyo(dictio,year)
            valor_prev = valor_anyo(dictio,prev_year)
          
            difabs = valor - valor_prev
            difpct= ((valor*100)/valor_prev)-100
            
            return(difabs,difpct)
    
    except KeyError as e:
        print('Issues in getting inc_anual: {error}.'.format(error=e))
        raise

### Funcion de reporte: totales

In [283]:
def reporte_totales (dictio):
    
    ancho=70
    sep='*'
    lin_sep = sep * ancho
    print(lin_sep)
    
    for aa in dictio:
        inctot,incpct = inc_anual(dic_total,aa)
        print(' Para el año {y}, la distancia  es {d:,} --> ({it:>+6,} , {ip:>+7.2f} %)'.format(y=aa, d=valor_anyo(dictio,aa),it=inctot, ip=incpct))
    print('\n')
    relleno = sep * (ancho-35)
    print(relleno, end=' ')
    print(' Esto hace un total de {t:>7,}'.format(t=valor_total(dictio)))
    print(lin_sep, '\n')

### Reporte mensual

In [369]:
def pintar_inc_mensual(year, month, val, incabs, incpct):
    escala=500 
    display=80
    
    start='| '
    rell1='*'
    rell2='-'
    
    mid1 ='{a}:'.format(a=year, m=month)
    
    arellenar = int((display*val)/escala)
    mid2 = rell1 * arellenar
    l1= len(mid2)
    l2= display-l1
    mid3 = rell2 * l2
   
    mid4 = '{v:>6} km, {ia:>+7.2f}, ({ip:+7.2f} %)'.format(v=val, ia=incabs, ip=incpct)

    end= ' |'
    
    linea= start+mid1+mid2+mid3+mid4+end
    print(linea)

def reporte_mensual (dictio, month):
    
    ancho=120
    sep='-'
    lin_sep = sep * ancho
    print(lin_sep)
    
    mid1=' Reporte del mes de {m}'.format(m=month)
    l1=len(mid1)
    mid2= ' '*(ancho-2-l1)
    
    lin_head = '|'+mid1+mid2+'|'
    print(lin_head)
    
    total=0
    i=0
    for aa in dictio:
        val, incabs, incpct = inc_mensual(dictio, aa, month)
        pintar_inc_mensual(aa, month, val, incabs, incpct)
        total = total+val
        i+=1
    
    line_space = '|' +' '*(ancho-2) +'|'
    print(line_space)
    
    avg=int(total/i)
    mid2 ='Distancia media en el mes de {mm} = {med} km.'.format(mm=month, med=avg)
    l2=len(mid2)
    mid1= ' '*(ancho-2-l2)
    
    lin_avg = '|'+mid1+ mid2+'|' 
    print(lin_avg)
    
    print(lin_sep)
    

def reporte_todos_meses(dictio):
    meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep','Oct', 'Nov', 'Dic']
    
    for mm in meses:
        reporte_mensual(dictio, mm)
        

# Programa principal

In [371]:
import pprint as pp

dic_total = leer_todos_ficheros()
#pp.pprint(dic_total)

reporte_totales(dic_total)

reporte_todos_meses(dic_total)

**********************************************************************
 Para el año 2016, la distancia  es 2,938 --> (+2,938 , +100.00 %)
 Para el año 2017, la distancia  es 2,919 --> (   -19 ,   -0.65 %)
 Para el año 2018, la distancia  es 3,308 --> (  +389 ,  +13.33 %)
 Para el año 2019, la distancia  es 3,338 --> (   +30 ,   +0.91 %)


***********************************  Esto hace un total de  12,503
********************************************************************** 

------------------------------------------------------------------------------------------------------------------------
| Reporte del mes de Ene                                                                                               |
| 2016:*******************************************-------------------------------------270.98 km, +270.98, (+100.00 %) |
| 2017:********************************------------------------------------------------203.02 km,  -67.96, ( -25.08 %) |
| 2018:****************************